# 3. AutoEncoder

In [11]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import numpy as np
import pickle

from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)
import category_encoders as ce

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

/Users/remuslum/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/remuslum/opt/anaconda3/lib/python3.8/site-packages/statsmodels/iolib/foreign.py:633: FutureWarning: In the future `np.long` will be defined as the corresponding NumPy scalar.
  _type_converters = {253 : np.long, 252 : int}


AttributeError: module 'numpy' has no attribute 'long'

In [2]:
df_train = pd.read_csv('/Users/remuslum/Downloads/bt4222_data/train_dataset.csv')
df_test = pd.read_csv('/Users/remuslum/Downloads/bt4222_data/test_dataset.csv')

In [3]:
df_train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V296,V297,V301,V303,V305,V307,V309,V310,V314,V320
0,2987001.0,0,86401.0,29.000,W,2755.0,404.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.00
1,2987002.0,0,86469.0,59.000,W,4663.0,490.0,150.0,visa,166.0,...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.00
2,2987003.0,0,86499.0,50.000,W,18132.0,567.0,150.0,mastercard,117.0,...,0.0,0.0,0.0,0.0,1.0,1758.00,0.0,354.0,0.0,0.00
3,2987004.0,0,86506.0,50.000,H,4497.0,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,1.0,1.0,0.00,0.0,0.0,0.0,0.00
4,2987005.0,0,86510.0,49.000,W,5937.0,555.0,150.0,visa,226.0,...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578076,3577534.0,0,15811030.0,67.505,C,5812.0,408.0,185.0,mastercard,224.0,...,0.0,0.0,0.0,1.0,1.0,0.00,0.0,0.0,0.0,0.00
578077,3577536.0,0,15811049.0,39.500,W,10444.0,225.0,150.0,mastercard,224.0,...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.00
578078,3577537.0,0,15811079.0,30.950,W,12037.0,595.0,150.0,mastercard,224.0,...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.00
578079,3577538.0,0,15811088.0,117.000,W,7826.0,481.0,150.0,mastercard,224.0,...,0.0,0.0,0.0,0.0,1.0,2903.50,117.0,669.5,669.5,0.00


In [9]:
object_columns = df_train.select_dtypes(include = ['object'])
object_columns

,ProductCD,card4,card6,P_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,mastercard,credit,gmail.com,T,T,T,M0,T,T,F,F,T
1,W,visa,debit,outlook.com,T,T,T,M0,F,F,F,F,F
2,W,mastercard,debit,yahoo.com,T,T,T,M0,T,F,F,F,T
3,H,mastercard,credit,gmail.com,T,T,T,M0,F,T,F,F,T
4,W,visa,debit,gmail.com,T,T,T,M1,F,T,F,F,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...
578076,C,mastercard,debit,gmail.com,T,T,T,M2,T,F,F,F,T
578077,W,mastercard,debit,gmail.com,T,F,F,M0,F,T,F,F,F
578078,W,mastercard,debit,gmail.com,T,F,F,M0,T,T,F,F,T
578079,W,mastercard,debit,aol.com,T,T,T,M0,F,T,F,F,T


In [ ]:
# Label encoding: M4
label_encoder = LabelEncoder()
df_train['M4_Encoded'] = label_encoder.fit_transform(df_train['M4'])

#Binary Encoding


# Target encoding: ProductCD, card4, card6, P_emaildomain
df_train['ProductCD_Encoded'] = label_encoder.fit_transform(df_train['ProductCD'])